In [ ]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [ ]:
!pip install transformers datasets


     |████████████████████████████████| 3.8 MB 3.9 MB/s 
     |████████████████████████████████| 312 kB 49.0 MB/s 
     |████████████████████████████████| 895 kB 49.7 MB/s 
     |████████████████████████████████| 6.5 MB 53.1 MB/s 
     |████████████████████████████████| 596 kB 50.2 MB/s 
     |████████████████████████████████| 67 kB 5.1 MB/s 
     |████████████████████████████████| 1.1 MB 47.1 MB/s 
     |████████████████████████████████| 212 kB 48.1 MB/s 
     |████████████████████████████████| 134 kB 43.4 MB/s 
     |████████████████████████████████| 127 kB 50.4 MB/s 
     |████████████████████████████████| 271 kB 51.5 MB/s 
     |████████████████████████████████| 94 kB 3.3 MB/s 
     |████████████████████████████████| 144 kB 52.7 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling

In [ ]:
!pip install deepcut

     |████████████████████████████████| 2.0 MB 7.5 MB/s 
     |████████████████████████████████| 462 kB 38.0 MB/s 


## **Warm-starting BERT2BERT for CNN/Dailymail**

***Note***: This notebook only uses a few training, validation, and test data samples for demonstration purposes. To fine-tune an encoder-decoder model on the full training data, the user should change the training and data preprocessing parameters accordingly as highlighted by the comments.


### **Load Data**

In [ ]:
with open('/content/tha.txt') as f: 
  lines = f.readlines()
  pairs = [line.split('\t')[:-1] for line in lines]

In [ ]:
import pandas as pd
import numpy as np
# import deepcut

train_df = pd.DataFrame(
    pairs, columns=["en", "th"]
)
#train_df['th'] = list(map(lambda sent: ' '.join(deepcut.tokenize(sent)), train_df['th']))
train_df['id'] = np.arange(len(train_df))
train_df.head()

,en,th,id
0,Hi.,สวัสดี,0
1,Run.,วิ่ง,1
2,Who?,ใคร?,2
3,Wow!,ว้าว!,3
4,Fire!,ยิง!,4


In [ ]:
!zip /content/checkpoint-1200.zip /content/checkpoint-1200

updating: content/checkpoint-1200/ (stored 0%)


In [ ]:
!ps -aux|grep python

root          36  0.1  0.0      0     0 ?        Z    11:49   0:04 [python3] <defunct>
root          37  0.0  0.1 163736 33448 ?        S    11:49   0:00 python3 /usr/local/bin/colab-fileshim.py
root          62  0.4  0.1 194928 52232 ?        Sl   11:49   0:16 /usr/bin/python2 /usr/local/bin/jupyter-notebook --ip="172.28.0.2" --port=9000 --FileContentsManager.root_dir="/" --MappingKernelManager.root_dir="/content"
root          88 45.7 40.8 44274576 10901052 ?   Ssl  11:50  27:38 /usr/bin/python3 -m ipykernel_launcher -f /root/.local/share/jupyter/runtime/kernel-732f4561-efa9-496c-b13b-a2a6deb7d042.json
root         110  0.3  0.0 129436 14864 ?        Sl   11:50   0:13 /usr/bin/python3 /usr/local/lib/python3.7/dist-packages/debugpy/adapter --for-server 43717 --host 127.0.0.1 --port 16224 --server-access-token 99c328b5897fd43e9d4132db423aeaf97543f9d25474c2d0f2ba8f5439b9182f
root         688  0.0  0.0  39200  6164 ?        S    12:50   0:00 /bin/bash -c ps -aux|grep python
root         

In [ ]:
!kill -9 88

In [ ]:
train_df.to_csv('df_.csv', index=False)

In [ ]:
import pandas as pd
train_df = pd.read_csv('/content/df_.csv')
train_df

,en,th,id
0,Hi.,สวัสดี,0
1,Run.,วิ่ง,1
2,Who?,ใคร?,2
3,Wow!,ว้าว!,3
4,Fire!,ยิง!,4
...,...,...,...
3712,"Before you put the knife away, please wipe it ...",ก่อนเก็บมีดเช็ดให้แห้งด้วย จะได้ไม่เป็นสนิม,3712
3713,"If I had known about your illness, I could hav...",ถ้าฉันรู้เรื่องที่คุณป่วย ฉันอาจจะไปเยี่ยมคุณท...,3713
3714,Some people say that you shouldn't check your ...,บางคนบอกว่า ไม่ควรตรวจสอบอีเมลเป็นสิ่งแรกในตอน...,3714
3715,Hand washing is recommended to prevent the spr...,แนะนำให้ล้างมือเพื่อป้องกันการแพร่กระจายของโรค...,3715


In [ ]:
import pyarrow as pa
import pyarrow.dataset as ds
import pandas as pd
from datasets import Dataset

### convert to Huggingface dataset
sample_dataset = Dataset(pa.Table.from_pandas(train_df))

In [ ]:
sample_dataset

Dataset({
    features: ['en', 'th', 'id'],
    num_rows: 3717
})

### **Data Preprocessing**


In [ ]:
# %%capture
# !pip install datasets==1.0.2
# !pip install transformers==4.2.1

In [ ]:
import datasets
import transformers

In [ ]:
tokenizer_input("ฉัน กิน ข้าว มาก apple และ สับปะรด?")

{'input_ids': [0, 59334, 6, 19094, 6, 40699, 6, 5597, 108787, 1494, 14990, 108389, 104033, 4041, 2851, 32, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [ ]:
tokenizer_input.additional_special_tokens

[]

Assigning <pad> to the pad_token key of the tokenizer
Adding <pad> to the vocabulary
Assigning <s> to the bos_token key of the tokenizer
Adding <s> to the vocabulary
Assigning </s> to the sep_token key of the tokenizer
Adding </s> to the vocabulary
Assigning <unk> to the unk_token key of the tokenizer
Assigning </s> to the eos_token key of the tokenizer
Assigning <s> to the cls_token key of the tokenizer
Assigning <mask> to the mask_token key of the tokenizer
Adding <mask> to the vocabulary


4

In [ ]:
from transformers import BertTokenizerFast
from transformers import XLMRobertaTokenizerFast, GPT2Tokenizer

tokenizer_input = XLMRobertaTokenizerFast.from_pretrained("xlm-roberta-base")
tokenizer_output = XLMRobertaTokenizerFast.from_pretrained("xlm-roberta-base")
#tokenizer_input = BertTokenizer.from_pretrained("bert-base-cased")
#tokenizer_output = GPT2Tokenizer.from_pretrained("gpt2")
# tokenizer.bos_token = tokenizer.cls_token
# tokenizer.eos_token = tokenizer.sep_token

# train_data = datasets.load_dataset("cnn_dailymail", "3.0.0", split="train")
# val_data = datasets.load_dataset("cnn_dailymail", "3.0.0", split="validation[:10%]")
data = sample_dataset.train_test_split(test_size = .15, seed = 999)
train_data = data['train']
val_data = data['test']
# train_data = sample_dataset
# val_data = sample_dataset
#val_data = datasets.load_dataset("cnn_dailymail", "3.0.0", split="validation[:10%]")

In [ ]:
str(1).split(" ")

['1']

In [ ]:
import torchvision.models as models
base_network = models.resnet101(pretrained=True)

Downloading: "https://download.pytorch.org/models/resnet101-63fe2227.pth" to /root/.cache/torch/hub/checkpoints/resnet101-63fe2227.pth


  0%|          | 0.00/171M [00:00<?, ?B/s]

In [ ]:
import torch
model = torch.load('/content/resnet101.pt')
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [ ]:
tokenizer_input

PreTrainedTokenizerFast(name_or_path='xlm-roberta-base', vocab_size=250002, model_max_len=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=False)})

In [ ]:
val_data

Dataset({
    features: ['en', 'th', 'id'],
    num_rows: 558
})

In [ ]:
train_data

Dataset({
    features: ['en', 'th', 'id'],
    num_rows: 3159
})

In [ ]:
# tokenizer_output.add_special_tokens({'pad_token':'<pad>','bos_token':'<s>','sep_token':'</s>',
#                                      'unk_token':'<unk>','eos_token':'</s>', 'cls_token':'<s>',
#                                      'mask_token':'<mask>'})
tokenizer_output.add_special_tokens({'pad_token':'<pad>'})

Assigning <pad> to the pad_token key of the tokenizer
Adding <pad> to the vocabulary


1

In [ ]:
tokenizer_output.get_added_vocab()

{'</s>': 50259, '<mask>': 50260, '<pad>': 50257, '<s>': 50258}

In [ ]:
batch_size= 4  # change to 16 for full training
encoder_max_length=65
decoder_max_length=65

def process_data_to_model_inputs(batch):
  # tokenize the inputs and labels
  inputs = tokenizer_input(batch["th"], add_special_tokens = True, padding="max_length", truncation=True, max_length=encoder_max_length)
  outputs = tokenizer_output(batch["en"],add_special_tokens = True, padding="max_length", truncation=True, max_length=decoder_max_length)

  batch["input_ids"] = inputs.input_ids
  batch["attention_mask"] = inputs.attention_mask
  batch["decoder_input_ids"] = outputs.input_ids
  batch["decoder_attention_mask"] = outputs.attention_mask
  batch["labels"] = outputs.input_ids.copy()


  # because BERT automatically shifts the labels, the labels correspond exactly to `decoder_input_ids`. 
  # We have to make sure that the PAD token is ignored
  batch["labels"] = [                                                                                 
        [-100 if token == tokenizer_output.pad_token_id else token for token in labels] for labels in batch["labels"]
    ]
  return batch

# only use 32 training examples for notebook - DELETE LINE FOR FULL TRAINING
#train_data = train_data.select(range(30))

train_data = train_data.map(
    process_data_to_model_inputs, 
    batched=True, 
    batch_size=batch_size, 
    remove_columns=["en", "th", "id"]
)
train_data.set_format(
    type="torch", columns=["input_ids", "attention_mask", "decoder_input_ids", "decoder_attention_mask", "labels"],
)


# only use 16 training examples for notebook - DELETE LINE FOR FULL TRAINING
#val_data = val_data.select(range(8))

val_data = val_data.map(
    process_data_to_model_inputs, 
    batched=True, 
    batch_size=batch_size, 
    remove_columns=["en", "th", "id"]
)
val_data.set_format(
    type="torch", columns=["input_ids", "attention_mask", "decoder_input_ids", "decoder_attention_mask", "labels"],
)

  0%|          | 0/790 [00:00<?, ?ba/s]

  0%|          | 0/140 [00:00<?, ?ba/s]

In [ ]:
train_data

Dataset({
    features: ['input_ids', 'attention_mask', 'decoder_input_ids', 'decoder_attention_mask', 'labels'],
    num_rows: 3159
})

### **Warm-starting the Encoder-Decoder Model**

In [ ]:
from transformers import EncoderDecoderModel

bert2bert = EncoderDecoderModel.from_encoder_decoder_pretrained('xlm-roberta-base', 'xlm-roberta-base', tie_encoder_decoder=True)
#bert2bert = EncoderDecoderModel.from_pretrained('/content/checkpoint-1200', tie_encoder_decoder=True)

Downloading:   0%|          | 0.00/1.04G [00:00<?, ?B/s]

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaModel: ['lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForCausalLM were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['roberta.encoder.layer.8.crossattention.output.dense.bias', 'roberta.encoder.layer.11.crossattention.output.dense.weight', 'roberta.encoder.layer.8.cro

In [ ]:
bert2bert.save_pretrained('./xlm-roberta2gpt')

Configuration saved in ./xlm-roberta2gpt/config.json
Model weights saved in ./xlm-roberta2gpt/pytorch_model.bin


In [ ]:
!pip install --upgrade git+https://github.com/google/flax.git

  Cloning https://github.com/google/flax.git to /tmp/pip-req-build-z9sutyfg
  Running command git clone -q https://github.com/google/flax.git /tmp/pip-req-build-z9sutyfg


In [ ]:
from transformers import EncoderDecoderModel

bert2bert = EncoderDecoderModel.from_encoder_decoder_pretrained('xlm-roberta-base', 'gpt2', tie_encoder_decoder=True)

loading configuration file https://huggingface.co/xlm-roberta-base/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/87683eb92ea383b0475fecf99970e950a03c9ff5e51648d6eee56fb754612465.dfaaaedc7c1c475302398f09706cbb21e23951b73c6e2b3162c1c8a99bb3b62a
Model config XLMRobertaConfig {
  "_name_or_path": "xlm-roberta-base",
  "architectures": [
    "XLMRobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "xlm-roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.17.0",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 250002
}

loading weights 

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

storing https://huggingface.co/gpt2/resolve/main/config.json in cache at /root/.cache/huggingface/transformers/fc674cd6907b4c9e933cb42d67662436b89fa9540a1f40d7c919d0109289ad01.7d2e0efa5ca20cef4fb199382111e9d3ad96fd77b849e1d4bed13a66e1336f51
creating metadata file for /root/.cache/huggingface/transformers/fc674cd6907b4c9e933cb42d67662436b89fa9540a1f40d7c919d0109289ad01.7d2e0efa5ca20cef4fb199382111e9d3ad96fd77b849e1d4bed13a66e1336f51
loading configuration file https://huggingface.co/gpt2/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/fc674cd6907b4c9e933cb42d67662436b89fa9540a1f40d7c919d0109289ad01.7d2e0efa5ca20cef4fb199382111e9d3ad96fd77b849e1d4bed13a66e1336f51
Model config GPT2Config {
  "_name_or_path": "gpt2",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_typ

Downloading:   0%|          | 0.00/523M [00:00<?, ?B/s]

storing https://huggingface.co/gpt2/resolve/main/pytorch_model.bin in cache at /root/.cache/huggingface/transformers/752929ace039baa8ef70fe21cdf9ab9445773d20e733cf693d667982e210837e.323c769945a351daa25546176f8208b3004b6f563438a7603e7932bae9025925
creating metadata file for /root/.cache/huggingface/transformers/752929ace039baa8ef70fe21cdf9ab9445773d20e733cf693d667982e210837e.323c769945a351daa25546176f8208b3004b6f563438a7603e7932bae9025925
loading weights file https://huggingface.co/gpt2/resolve/main/pytorch_model.bin from cache at /root/.cache/huggingface/transformers/752929ace039baa8ef70fe21cdf9ab9445773d20e733cf693d667982e210837e.323c769945a351daa25546176f8208b3004b6f563438a7603e7932bae9025925
All model checkpoint weights were used when initializing GPT2LMHeadModel.

Some weights of GPT2LMHeadModel were not initialized from the model checkpoint at gpt2 and are newly initialized: ['h.7.crossattention.c_proj.bias', 'h.3.crossattention.c_proj.weight', 'h.7.crossattention.masked_bias', 'h

In [ ]:
bert2bert

EncoderDecoderModel(
  (encoder): XLMRobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(250002, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): Lay

In [ ]:
for param in bert2bert.encoder.parameters():
    param._trainable = False
# for param in bert2bert.decoder.parameters():
#     param._trainable = False

In [ ]:
# set special tokens
bert2bert.config.decoder_start_token_id = tokenizer_output.bos_token_id
bert2bert.config.eos_token_id = tokenizer_input.eos_token_id
bert2bert.config.pad_token_id = tokenizer_input.pad_token_id
# bert2bert.config.eos_token_id = tokenizer_output.eos_token_id
# bert2bert.config.pad_token_id = tokenizer.pad_token_id
# bert2bert.config.decoder_eos_token_id = tokenizer.eos_token_id
# bert2bert.config.decoder_pad_token_id = tokenizer.pad_token_id

# sensible parameters for beam search
bert2bert.config.vocab_size = bert2bert.config.encoder.vocab_size
bert2bert.config.max_length = 65
#bert2bert.config.min_length = 2
bert2bert.config.no_repeat_ngram_size = 2
bert2bert.config.early_stopping = True
bert2bert.config.length_penalty = 12.0
bert2bert.config.num_beams = 20
# bert2bert.config.num_return_sequences = 5

# model.config.decoder_start_token_id = tokenizer.cls_token_id
# model.config.pad_token_id = tokenizer.pad_token_id
# model.config.vocab_size = model.config.decoder.vocab_size

### **Fine-Tuning Warm-Started Encoder-Decoder Models**

For the `EncoderDecoderModel` framework, we will use the `Seq2SeqTrainingArguments` and the `Seq2SeqTrainer`. Let's import them.

In [ ]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer

In [ ]:
!pip install rouge_score

Also, we need to define a function to correctly compute the ROUGE score during validation. ROUGE is a much better metric to track during training than only language modeling loss.

In [ ]:
# load rouge for validation
import datasets
rouge = datasets.load_metric("rouge")

def compute_metrics(pred):
    labels_ids = pred.label_ids
    pred_ids = pred.predictions

    # all unnecessary tokens are removed
    pred_str = tokenizer_output.batch_decode(pred_ids, skip_special_tokens=True)
    labels_ids[labels_ids == -100] = tokenizer_input.pad_token_id
    label_str = tokenizer_input.batch_decode(labels_ids, skip_special_tokens=True)

    rouge_output = rouge.compute(predictions=pred_str, references=label_str, rouge_types=["rouge2"])["rouge2"].mid

    return {
        "rouge2_precision": round(rouge_output.precision, 4),
        "rouge2_recall": round(rouge_output.recall, 4),
        "rouge2_fmeasure": round(rouge_output.fmeasure, 4),
    }

Downloading:   0%|          | 0.00/2.16k [00:00<?, ?B/s]

Cool! Finally, we start training.

In [ ]:
# train_data, 

In [ ]:
# # https://medium.com/@pierre_guillou/nlp-how-to-add-a-domain-specific-vocabulary-new-tokens-to-a-subword-tokenizer-already-trained-33ab15613a41
# import torch
# from transformers import BertTokenizer, BertModel

# tokenizer = BertTokenizer.from_pretrained("bert-base-cased")
# model = BertModel.from_pretrained("bert-base-cased")

# print(len(tokenizer))  # 28996
# tokenizer.add_tokens(["NEW_TOKEN"])
# print(len(tokenizer))  # 28997

# model.resize_token_embeddings(len(tokenizer)) 
# # The new vector is added at the end of the embedding matrix

# print(model.embeddings.word_embeddings.weight[-1, :])
# # Randomly generated matrix

# model.embeddings.word_embeddings.weight[-1, :] = torch.zeros([model.config.hidden_size])

# print(model.embeddings.word_embeddings.weight[-1, :])
# # outputs a vector of zeros of shape [768]


# # Let's increase the vocabulary of Bert model and tokenizer
# new_tokens = [token1, token2, token3, ..., token n]
# num_added_toks = tokenizer.add_tokens(new_tokens)
# print('We have added', num_added_toks, 'tokens')
# # Notice: resize_token_embeddings expect to receive the full size of the new vocabulary, i.e., the length of the tokenizer.
# model.resize_token_embeddings(len(tokenizer))

In [ ]:
# set training arguments - these params are not really tuned, feel free to change
training_args = Seq2SeqTrainingArguments(
    output_dir="/content/drive/MyDrive/FaiFolder",
    evaluation_strategy="steps",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    logging_steps=2000,  # set to 1000 for full training
    save_steps=2000,  # set to 500 for full training
    eval_steps=2000,  # set to 8000 for full training
    warmup_steps=3000,  # set to 2000 for full training
    #max_steps=16, # delete for full training
    overwrite_output_dir=True,
    save_total_limit=3,
    num_train_epochs= 500,
    load_best_model_at_end=True
    # fp16=True,
    # num_return_sequences = 3
)

# instantiate trainer
trainer = Seq2SeqTrainer(
    model=bert2bert,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_data,
    eval_dataset=val_data,
)
trainer.train()

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 3159
  Num Epochs = 500
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 197500
/usr/local/lib/python3.7/dist-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:531: FutureWarning: Version v4.12.0 intr

Step,Training Loss,Validation Loss,Rouge2 Precision,Rouge2 Recall,Rouge2 Fmeasure
2000,4.818800,3.195259,0.108900,0.063700,0.076000
4000,2.522600,2.534557,0.174500,0.175700,0.171400
6000,1.520100,3.245587,0.158500,0.157600,0.154700
8000,1.900600,5.963623,0.051200,0.134100,0.067900
10000,2.060600,11.598143,0.011700,0.096500,0.019800
12000,1.597400,16.170710,0.003800,0.023000,0.006000
14000,1.268000,13.150466,0.007200,0.062100,0.012700
16000,1.073500,13.881927,0.004800,0.043200,0.008600
18000,0.981100,13.788748,0.005900,0.054800,0.010500
20000,0.889700,14.157415,0.005100,0.043400,0.009100


***** Running Evaluation *****
  Num examples = 558
  Batch size = 8
Saving model checkpoint to /content/drive/MyDrive/FaiFolder/checkpoint-2000
Configuration saved in /content/drive/MyDrive/FaiFolder/checkpoint-2000/config.json
Model weights saved in /content/drive/MyDrive/FaiFolder/checkpoint-2000/pytorch_model.bin
/usr/local/lib/python3.7/dist-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:531: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.12.0. The decoder_input_ids are now created based on the labels, no need to pass them yourself anymore.
  warnings.warn(DEPRECATION_WARNING, FutureWarning)
***** Running Evaluation *****
  Num examples = 558
  Batch size = 8
Saving model checkpoint to /content/drive/MyDrive/FaiFolder/checkpoin

Step,Training Loss,Validation Loss,Rouge2 Precision,Rouge2 Recall,Rouge2 Fmeasure
2000,4.818800,3.195259,0.108900,0.063700,0.076000
4000,2.522600,2.534557,0.174500,0.175700,0.171400
6000,1.520100,3.245587,0.158500,0.157600,0.154700
8000,1.900600,5.963623,0.051200,0.134100,0.067900
10000,2.060600,11.598143,0.011700,0.096500,0.019800
12000,1.597400,16.170710,0.003800,0.023000,0.006000
14000,1.268000,13.150466,0.007200,0.062100,0.012700
16000,1.073500,13.881927,0.004800,0.043200,0.008600
18000,0.981100,13.788748,0.005900,0.054800,0.010500
20000,0.889700,14.157415,0.005100,0.043400,0.009100


Saving model checkpoint to /content/drive/MyDrive/FaiFolder/checkpoint-44000
Configuration saved in /content/drive/MyDrive/FaiFolder/checkpoint-44000/config.json
Model weights saved in /content/drive/MyDrive/FaiFolder/checkpoint-44000/pytorch_model.bin
Deleting older checkpoint [/content/drive/MyDrive/FaiFolder/checkpoint-40000] due to args.save_total_limit
/usr/local/lib/python3.7/dist-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:531: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.12.0. The decoder_input_ids are now created based on the labels, no need to pass them yourself anymore.
  warnings.warn(DEPRECATION_WARNING, FutureWarning)
***** Running Evaluation *****
  Num examples = 558
  Batch size = 8
Saving model checkpoint to /

RuntimeError: ignored

In [ ]:
!zip -r /content/checkpoint-11800.zip /content/checkpoint-11800

  adding: content/checkpoint-11800/ (stored 0%)
  adding: content/checkpoint-11800/trainer_state.json (deflated 87%)
  adding: content/checkpoint-11800/pytorch_model.bin (deflated 7%)
  adding: content/checkpoint-11800/rng_state.pth (deflated 27%)
  adding: content/checkpoint-11800/config.json (deflated 81%)
  adding: content/checkpoint-11800/optimizer.pt (deflated 7%)
  adding: content/checkpoint-11800/scheduler.pt (deflated 49%)
  adding: content/checkpoint-11800/training_args.bin (deflated 49%)


### **Evaluation**

Awesome, we finished training our dummy model. Let's now evaluated the model on the test data. We make use of the dataset's handy `.map()` function to generate a summary of each sample of the test data.

In [ ]:
import datasets
from transformers import BertTokenizer, EncoderDecoderModel
import torch
from transformers import XLMRobertaTokenizerFast

#tokenizer = XLMRobertaTokenizerFast.from_pretrained("xlm-roberta-base")
model = EncoderDecoderModel.from_pretrained("/content/drive/MyDrive/FaiFolder/checkpoint-4000")
#model = EncoderDecoderModel.from_encoder_decoder_pretrained('bert-base-multilingual-cased', 'bert-base-cased')
#model.to("cuda")
#model.to(torch.device("cuda:0"))

# test_data = datasets.load_dataset("cnn_dailymail", "3.0.0", split="test")
test_data = data['train']

# only use 16 training examples for notebook - DELETE LINE FOR FULL TRAINING
test_data = test_data.select(range(16))
# test_data = sample_dataset

batch_size = 8  # change to 64 for full evaluation

# map data correctly
def generate_summary(batch):
    # Tokenizer will automatically set [BOS] <text> [EOS]
    # cut off at BERT max length 512
    inputs = tokenizer_input(batch["th"], padding="max_length", truncation=True, max_length=150, return_tensors="pt")
    print(inputs)
    input_ids = inputs.input_ids
    input_ids = inputs.input_ids
    attention_mask = inputs.attention_mask

    outputs = model.generate(input_ids, attention_mask=attention_mask)

    # all special tokens including will be removed
    output_str = tokenizer_output.batch_decode(outputs, skip_special_tokens=True)

    batch["pred"] = output_str

    return batch

results = test_data.map(generate_summary, batched=True, batch_size=batch_size, remove_columns=["th"])

pred_str = results["pred"]
label_str = results["en"]

rouge_output = rouge.compute(predictions=pred_str, references=label_str, rouge_types=["rouge2"])["rouge2"].mid

print(rouge_output)

ModuleNotFoundError: ignored

The fully trained *BERT2BERT* model is uploaded to the 🤗model hub under [patrickvonplaten/bert2bert_cnn_daily_mail](https://huggingface.co/patrickvonplaten/bert2bert_cnn_daily_mail). 

The model achieves a ROUGE-2 score of **18.22**, which is even a little better than reported in the paper.

For some summarization examples, the reader is advised to use the online inference API of the model, [here](https://huggingface.co/patrickvonplaten/bert2bert_cnn_daily_mail).

In [ ]:
pred_str, label_str

(["I I I'm Tom You She My It We  They TheI He There II If Mary S M",
  "I We Please They You He It She don dis Why There Tom This Our Let Don Most Do' The",
  "That This It Tom One There You Some might Could The She Something Of Those Bar' Three No Here Hand",
  'I I I She can You They My Can We am Tom swim It',
  "Tom Don I Please My Stop Let' Could Do Exc Take Put Didt Turn There Where Smo It Keep",
  'Everyone Everything Tom This Life is She Something Of The He French Japan I Answer Mom Mary My School There Thanks',
  'I have My Have We All Do finished You Tom am She How Thanks He They French II already Let Are',
  "Please Don I Tom Stop Let' My Where Do Go Could Put Theret Come Take Why You Exc Turn",
  'Tom I He wanted Mary She My suspect Hi The We was to You Bostoned It They Let Did Who',
  "We I You They He Please There It She don Tom Let Our Don This The' Why Do dis we",
  'He She Tom Mary This There Of You Everyone The is One English I All A French Only Has a Who',
  'I I I pr

In [ ]:
test_data[0]

{'en': "I'm very hungry now.", 'id': 1385, 'th': 'ฉันหิวมากตอนนี้'}